## Challenge code
 
 In the code below, you are given a function called `is_unsafe`. **You must complete this function** by coming up with a way — you are given total freedom, from making a variational circuit to finding an analytical solution — to determine if the given values of $\alpha$ and $\beta$ values are $\epsilon$-unsafe.

 ![img](images/operator_encoder.jpeg)

 ![img](images/tel-bell.png)
 
 ### Inputs
 
 As input to this problem, you are given a `list(float)` containing the values of $\alpha$, $\beta$, and $\epsilon$, in that order.
 
 ### Output
 
 This code must output a boolean — `True` or `False` — corresponding to whether the values of $\alpha$ and $\beta$ are $\epsilon$-unsafe.
 For example, if you determine that the given values of $\alpha$ and $\beta$ *aren't* $\epsilon$-unsafe, your code must output `False`. 
 
 If your solution is correct, the output will be `"Correct!"` Otherwise, you will receive a `"Wrong answer"` prompt.
 
 Good luck!
 ### Imports
 The cell below specifies the libraries you should use in this challenge. Run the cell to import the libraries. ***Do not modify the cell.***

In [1]:
import json
import pennylane as qml
from pennylane import numpy as np

### Code
 Complete the code below. Note that during QHack, some sections were not editable. We've marked those sections accordingly here, but you can still edit them if you wish.

In [ ]:
# Uneditable section #
def U_psi(theta):
    """
    Quantum function that generates |psi>, Zenda's state wants to send to Reece.

    Args:
        theta (float): Parameter that generates the state.

    """
    qml.Hadamard(wires = 0)
    qml.CRX(theta, wires = [0,1])
    qml.CRZ(theta, wires = [0,1])

In [ ]:
#### this could be a solution
#### BUT the operator RX(beta) @ RX(beta) @ RZ(alpha) @ RZ(alpha) is not a valid observable
#### it is not hermitian

def is_unsafe_0(alpha, beta, epsilon):

    dev = qml.device("default.qubit", wires=2)
    @qml.qnode(dev)
    def expectation(theta):
        U_psi(theta)
    
        # expectation value of the state |psi> with respect to the operator
        # (RX(beta) @ RX(beta)) @ (RZ(alpha) @ RZ(alpha))
        # (the encoding)

        # op_wire_1 = qml.prod(qml.RX(beta, 0), qml.RZ(alpha, 0))
        # op_wire_2 = qml.prod(qml.RX(beta, 1), qml.RZ(alpha, 1))
        # return qml.expval(op_wire_1 @ op_wire_2)
        return qml.expval((qml.RX(beta, wires=0) @ qml.RX(beta, wires=0)) @ 
                          (qml.RZ(alpha, wires=0) @ qml.RZ(alpha, wires=0)))
      
    thetas = np.arange(-np.pi, np.pi, 0.1)
    for theta in thetas:
        # check if abs(expectation)**2 is >= 1-epsilon
        if abs(expectation(theta))**2 >= 1 - epsilon:
            return True
        return False

in0 = [0.1, 0.2, 0.3]
try:
    print(is_unsafe_0(*in0))
except Exception as e:
    print("Error:", e)

# in the equation we need to consider also the imaginary part

Error: Measurement expval(RX(0.2, wires=[0]) @ RX(0.2, wires=[0]) @ RZ(0.1, wires=[0]) @ RZ(0.1, wires=[0])) not accepted for analytic simulation on default.qubit.


In [ ]:

def is_unsafe(alpha, beta, epsilon):
    """
    Boolean function that we will use to know if a set of parameters is unsafe.

    Args:
        alpha (float): parameter used to encode the state.
        beta (float): parameter used to encode the state.
        epsilon (float): unsafe-tolerance.

    Returns:
        (bool): 'True' if alpha and beta are epsilon-unsafe coefficients. 'False' in the other case.

    """
    # End of uneditable section #
    # instead we are going to compute the matrix 
    # and just compute the matrix multiplication
    o = qml.matrix((qml.RX(beta, 0) @ qml.RX(beta, 1)) @
                   (qml.RZ(alpha, 0) @ qml.RZ(alpha, 1)))
    
    bra_0 = np.array([1, 0, 0, 0]) # |00>
    ket_0 = bra_0.T
    thetas = np.arange(-np.pi, np.pi, 0.1)
    for theta in thetas:
        U = qml.matrix(U_psi, wire_order=[0,1])(theta)
        U_dagger = qml.matrix(qml.adjoint(U_psi), wire_order=[0,1])(theta)

        # just matrix multiplication
        exp_val = bra_0 @ U_dagger @ o @ U @ ket_0
        if abs(exp_val)**2 >= 1 - epsilon:
            return True
    return False


in0 = [0.1, 0.2, 0.3]
print(is_unsafe(*in0))

True


These functions are responsible for testing the solution. You will need to run the cell below. ***Do not modify the cell.***

In [23]:
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    output = is_unsafe(*ins)
    return str(output)

def check(solution_output: str, expected_output: str) -> None:
    
    def bool_to_int(string):
        if string == "True":
            return 1
        return 0

    solution_output = bool_to_int(solution_output)
    expected_output = bool_to_int(expected_output)
    assert solution_output == expected_output, "The solution is not correct."

### Test cases
 Running the cell below will load the test cases. ***Do not modify the cell***.
 - input: [0.1, 0.2, 0.3]
 	+ expected output: True
 - input: [1.1, 1.2, 0.3]
 	+ expected output: False
 - input: [1.1, 1.2, 0.4]
 	+ expected output: True
 - input: [0.5, 1.9, 0.7]
 	+ expected output: True
 - input: [0.5, 1.9, 0.3]
 	+ expected output: False
 - input: [0.5, 1.1, 0.5]
 	+ expected output: True
 - input: [0.1, 1.1, 0.2]
 	+ expected output: False
 - input: [0.2, 1.0, 0.3]
 	+ expected output: True

In [24]:
test_cases = [['[0.1, 0.2, 0.3]', 'True'], ['[1.1, 1.2, 0.3]', 'False'], ['[1.1, 1.2, 0.4]', 'True'], ['[0.5, 1.9, 0.7]', 'True'], ['[0.5, 1.9, 0.3]', 'False'], ['[0.5, 1.1, 0.5]', 'True'], ['[0.1, 1.1, 0.2]', 'False'], ['[0.2, 1.0, 0.3]', 'True']]

### Solution testing
 Once you have run every cell above, including the one with your code, the cell below will test your solution. Run the cell. If you are correct for all of the test cases, it means your solutions is correct. Otherwise, you need to double check your work. ***Do not modify the cell below.***

In [25]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[0.1, 0.2, 0.3]'...
Correct!
Running test case 1 with input '[1.1, 1.2, 0.3]'...
Correct!
Running test case 2 with input '[1.1, 1.2, 0.4]'...
Correct!
Running test case 3 with input '[0.5, 1.9, 0.7]'...
Correct!
Running test case 4 with input '[0.5, 1.9, 0.3]'...
Correct!
Running test case 5 with input '[0.5, 1.1, 0.5]'...
Correct!
Running test case 6 with input '[0.1, 1.1, 0.2]'...
Correct!
Running test case 7 with input '[0.2, 1.0, 0.3]'...
Correct!
